In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score
import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.search import ConcurrencyLimiter
from ray.tune.schedulers import ASHAScheduler

# Initialize Ray
ray.init()

# Load data from CSV files
train_df = pd.read_csv("train.csv", header=None)
valid_df = pd.read_csv("validation.csv", header=None)
test_df = pd.read_csv("test.csv", header=None)

# Separate features and target variables for training, validation, and testing datasets
x_train, y_train = train_df.iloc[:, 1:], train_df.iloc[:, 0]
x_valid, y_valid = valid_df.iloc[:, 1:], valid_df.iloc[:, 0]
x_test, y_test = test_df.iloc[:, 1:], test_df.iloc[:, 0]

# Combine training and validation data for final training
X_combined = pd.concat([x_train, x_valid])
y_combined = pd.concat([y_train, y_valid])

# Put large datasets in the Ray object store to manage memory usage
x_train_ref = ray.put(x_train)
y_train_ref = ray.put(y_train)
x_valid_ref = ray.put(x_valid)
y_valid_ref = ray.put(y_valid)


# Define the objective function for Ray Tune
def objective(config):
    # Define the parameters for the XGBoost model
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "booster": "gbtree",
        "max_depth": 3,  # Fixed max_depth to 3
        "learning_rate": config["learning_rate"],
        "n_estimators": int(config["n_estimators"]),
        "min_child_weight": int(config["min_child_weight"]),
        "gamma": config["gamma"],
        "subsample": config["subsample"],
        "colsample_bytree": config["colsample_bytree"],
    }

    model = xgb.XGBClassifier(**param)

    # Retrieve datasets from the Ray object store
    x_train = ray.get(x_train_ref)
    y_train = ray.get(y_train_ref)
    x_valid = ray.get(x_valid_ref)
    y_valid = ray.get(y_valid_ref)

    # Train the model and measure the training time
    model.fit(x_train, y_train)
    preds = model.predict(x_valid)
    accuracy = accuracy_score(y_valid, preds)

    # Report accuracy back to Ray Tune
    train.report({"mean_accuracy": accuracy})


# Define the search space for hyperparameter tuning
search_space = {
    "learning_rate": tune.uniform(0.01, 0.3),
    "n_estimators": tune.randint(100, 1000),
    "min_child_weight": tune.randint(1, 10),
    "gamma": tune.uniform(1e-8, 1.0),
    "subsample": tune.uniform(0.5, 1.0),
    "colsample_bytree": tune.uniform(0.5, 1.0),
}

# Define the Optuna search algorithm for hyperparameter tuning
algo = OptunaSearch()

# Limit the number of concurrent trials to avoid excessive resource usage
algo = ConcurrencyLimiter(algo, max_concurrent=94)

# Define the ASHAScheduler for early stopping of underperforming trials
scheduler = ASHAScheduler(max_t=1000, grace_period=1, reduction_factor=2)


# Function to create directory names for each trial
def trial_dirname_creator(trial):
    return f"trial_{trial.trial_id}"


# Define the Tuner for Ray Tune
tuner = tune.Tuner(
    objective,
    tune_config=tune.TuneConfig(
        metric="mean_accuracy",
        mode="max",
        search_alg=algo,
        scheduler=scheduler,
        num_samples=1000,  # Number of trials
        trial_dirname_creator=trial_dirname_creator,
    ),
    param_space=search_space,
)

# Run the hyperparameter tuning
results = tuner.fit()

# Extract the best result from the tuning process
best_result = results.get_best_result(metric="mean_accuracy", mode="max")
best_config = best_result.config
best_checkpoint = best_result.checkpoint

# Get a dataframe of all results
df = results.get_dataframe()

2024-07-25 09:19:45,513	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/objective_2024-07-25_08-46-13' in 0.8622s.
2024-07-25 09:19:45,705	INFO tune.py:1041 -- Total run time: 2012.12 seconds (2011.04 seconds for the tuning loop).


In [2]:
df.to_csv("results.csv")

In [3]:
best_config

{'learning_rate': 0.293064528332077,
 'n_estimators': 991,
 'min_child_weight': 7,
 'gamma': 0.3438511882472097,
 'subsample': 0.9654982494169886,
 'colsample_bytree': 0.7673390545515997}

In [4]:
import json
with open('best_config.json', 'w') as json_file:
    json.dump(best_config, json_file, indent=4)

In [5]:
param = {
    "verbosity": 0,
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "booster": "gbtree",
    "max_depth": 3,
    "learning_rate": best_config["learning_rate"],
    "n_estimators": int(best_config["n_estimators"]),
    "min_child_weight": int(best_config["min_child_weight"]),
    "gamma": best_config["gamma"],
    "subsample": best_config["subsample"],
    "colsample_bytree": best_config["colsample_bytree"],
}

# Train the model
model = xgb.XGBClassifier(**param)
model.fit(x_train, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7673390545515997, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None,
              gamma=0.3438511882472097, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.293064528332077,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=7, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=991, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [6]:
preds = model.predict(x_valid)
accuracy = accuracy_score(y_valid, preds)
print(accuracy)

0.9012528605060832


In [7]:
model.save_model('xgboost_model.json')

In [8]:
loaded_model = xgb.XGBClassifier()
loaded_model.load_model('xgboost_model.json')

In [17]:
from PIL import Image
import numpy as np

image_name = 'image-9.jpg'

# Open the image
image = Image.open(image_name)

# Convert the image to RGB (just in case it's in a different mode)
image_rgb = image.convert('RGB')

# Convert the image to a NumPy array
image_array = np.array(image_rgb)

# Reshape the image array to have shape (height * width, 3)
height, width, _ = image_array.shape
pixels = image_array.reshape(-1, 3)

# Classify each pixel
# Predict the class for each pixel
predictions = loaded_model.predict(pixels)

# Recreate the image
# Create a new image array where pixels classified as 1 are set to red
output_array = image_array.copy()
output_array = output_array.reshape(-1, 3)
output_array[predictions == 1] = [255, 0, 0]  # Red color

# Reshape the output array back to the original image shape
output_array = output_array.reshape(height, width, 3)

# Convert the array to an image
output_image = Image.fromarray(output_array.astype("uint8"))

# Save or display the output image
output_image.save(f"output-{image_name}.png")
output_image.show()


display-im6.q16: unable to open X server `' @ error/display.c/DisplayImageCommand/412.
